# Tutorial for Searching a STAC Catalog

In this tutorial you will learn how to connect to a STAC API and search various datasets available for query. You will also search for specific items using a range of query parameters.  

We will use the Earth Search API developed by Element84 for satellite datasets available on AWS S3 Storage (note that only those datasets that have a STAC catalog are served through this API). 

We will use the PySTAC pakcage for this tutorial

In [ ]:
from pystac_client import Client

In [ ]:
api_url = "https://earth-search.aws.element84.com/v1"

In [ ]:
client = Client.open(api_url)

## Find Collections

First, we would like to see what collections are available from this API. 

In [ ]:
collections = client.get_collections()

In [ ]:
for collection in collections:
    print(collection)

In order to get more information about a specific collection, you can use `get_collection` function:

In [ ]:
client.get_collection("sentinel-1-grd")

## Search Items

Let's use Leafmap to select a point where we are interested to find a satellite imagery

In [ ]:
import leafmap

In [ ]:
m = leafmap.Map(center=[42.250809, -71.822833], zoom=16, height="800px")
m

Pan and zoom the map to find an area of interest, then use the tools on the top left of the map to select a point on the map. 


In [ ]:
if m.user_rois is not None:
    point = m.user_rois['features'][0]['geometry']
else:
    point = dict(type="Point", coordinates=(42.250809, -71.822833))

In [ ]:
s2_collection = "sentinel-2-l2a"  # Sentinel-2, Level 2A, Cloud Optimized GeoTiffs (COGs)

In [ ]:
search = client.search(
    collections=[s2_collection],
    intersects=point,
    max_items=12,
)

In [ ]:
print(search.matched())

In [ ]:
items = search.item_collection()

In [ ]:
len(items)

Let's investigate an item

In [ ]:
print(items[0].datetime)

In [ ]:
print(items[0].geometry)

## Query Metadata

Items in STAC catalog have much more metadata (than location and time) that you can query and only return results that match your query parameters. 
One such metadata property is cloud cover percentage. Cloud cover is recorded in the metadata named `eo:cloud_cover`, and it ranges from 0 to 1. In the following, we are going to find scenes that only have a cloud cover of less than 15%.

In [ ]:
search = client.search(
    collections=[s2_collection],
    intersects=point,
    datetime="2023-01-01/2023-09-22"
)

In [ ]:
print(search.matched())

In [ ]:
search = client.search(
    collections=[s2_collection],
    intersects=point,
    query=["eo:cloud_cover<5"],
    max_items=10
)

In [ ]:
print(search.matched())

In [ ]:
items = search.item_collection()

In [ ]:
len(items)

In [ ]:
items[0]

In [ ]:
items.save_object("search.json")

## Access Assets

In [ ]:
assets = items[2].assets  # first item's asset dictionary
print(assets.keys())

In [ ]:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

In [ ]:
print(assets["thumbnail"].href)

In [ ]:
import requests
img_data = requests.get(assets["thumbnail"].href).content

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import io
plt.figure(figsize=(10, 10))
plt.imshow(Image.open(io.BytesIO(img_data)))

In [ ]:
import rioxarray
nir_href = assets["nir"].href
nir = rioxarray.open_rasterio(nir_href)
nir

In [ ]:
nir[0,1400:2000,4200:5000].mean()

In [ ]:
nir[0,1500:2000,6200:7000].rio.to_raster("nir_subset.tif")
